In [8]:
import pandas as pd
import os
import json
from pymongo import MongoClient
DATA = '../../../../data/'


In [9]:
def decorateRow(row):
    #['HEX_600', 'HEX_weighted_average_distance', 'HEX_weighted_mean_home_distance', 'HEX_weighted_max_home_distance', 'HEX_weighted_median_home_distance', 'HEX_weighted_radius_of_gyration', 'HEX_weighted_num_reporting_2015', 'HEX_weighted_HEX_coverage', 'HEX_weighted_subject_coverage', 'HEX_weighted_subject_hhi', 'HEX_total_reporting', 'HEX_total_user']
    record = row[1:].to_frame().transpose()
    return record

In [10]:
#Connect to client
client = MongoClient('localhost', 27017)
db = client['boston311']
hexagon_coll = db['bos311Hex']
# Drop it if it exsits.
hexagon_coll.drop() 

In [11]:
data_all_types = pd.read_csv(DATA + "hex_variables_all/poverty_added_01_hex_variables_non_gov_regardless_of_reporting_frequency_20210209.csv", sep='\t', header=0, index_col = 0)
data_all_types.columns

Index(['HEX_600', 'HEX_weighted_average_distance',
       'HEX_weighted_mean_home_distance', 'HEX_weighted_max_home_distance',
       'HEX_weighted_median_home_distance', 'HEX_weighted_radius_of_gyration',
       'HEX_weighted_num_reporting_2015', 'HEX_weighted_HEX_coverage',
       'HEX_weighted_subject_coverage', 'HEX_weighted_subject_hhi',
       'HEX_total_reporting', 'HEX_total_user',
       'HEX_normalized_total_311_per_100',
       'HEX_num_311_including_government_users', 'HEX_total_population',
       'HEX_income_below_poverty', 'HEX_below_poverty_unemployed',
       'HEX_above_poverty_unemployed', 'HEX_total_housing_units',
       'HEX_Owner_occupied_housing_units', 'HEX_Renter_occupied_housing_units',
       'HEX_less_than_bacholar', 'HEX_estimate_total_for_degree',
       'HEX_White_population', 'HEX_Black_or_African_American_population',
       'HEX_American_Indian_and_Alaska_Native_population',
       'HEX_Asian_population',
       'HEX_Native_Hawaiian_and_Other_Pacific_I

In [12]:
# all subjects
#Initialize collection
data_all_report_types = pd.read_csv(DATA + "hex_variables_all/poverty_added_01_hex_variables_non_gov_regardless_of_reporting_frequency_20210209.csv", sep='\t', header=0, index_col = 0)
#Adding object for each hexagon
#Object has 2 values: hexagon number and array of objects for data from each user type and frequency on hexagon
for index, row in data_all_report_types.iterrows():
    hexagon_coll.insert_one({
        'HEX_600': row.HEX_600,
        'results': []
    })
#Populating with data
all_report_type_dir = DATA + "hex_variables_all/"
for filename in os.listdir(all_report_type_dir):
    if filename.endswith(".csv"):
        data_all_report_types= pd.read_csv(os.path.join(all_report_type_dir, filename), sep='\t', header=0, index_col=0)
        data_all_report_types['report_type'] = "all"
        for index, row in data_all_report_types.iterrows():
            # Append new object from file to existing hexagon's array
            hexagon_coll.update_one({
                'HEX_600': row.HEX_600,
            },
            {
                '$push': 
                {
                    'results': json.loads(decorateRow(row).to_json(orient='records'))[0]
                }
            })

In [13]:
# by types
type_dir = DATA + "hex_variables_of_top_five_types_for_viz_system_20220923/"
for report_type in os.listdir(type_dir):
    if report_type!=".DS_Store":
        directory = type_dir + report_type + "/"
       
        for filename in os.listdir(directory):
            if filename.endswith(".csv"):
                frequency = "all"
                if filename.startswith("01"):
                    user_type = "non_gov"
                elif filename.startswith("02"):
                    user_type = "non_gov_unsure"
                elif filename.startswith("03"):
                    user_type = "all"
                elif filename.startswith("04"):
                    user_type = "non_gov"
                    frequency = "heavy"
                elif filename.startswith("05"):
                    user_type = "non_gov_unsure"
                    frequency = "heavy"
                elif filename.startswith("06"):
                    user_type = "all"
                    frequency = "heavy"
                
                data_per_type = pd.read_csv(os.path.join(directory, filename), header=0, index_col = 0)
                data_per_type.rename(columns={"HEX":"HEX_600","HEX_total_number_of_reports":"HEX_total_reporting"}, inplace=True)

                data_per_type['report_type'] = report_type
                data_per_type['user_type'] = user_type
                data_per_type['frequency'] = frequency
                data = data_per_type.merge(data_all_report_types, on = 'HEX_600')
                data = data.loc[:, ~data.columns.str.contains('_y$')]
                data.columns = data.columns.str.replace(r'_x$', '')
                for index, row in data.iterrows():
                    # Append new object from file to existing hexagon's array
                    hexagon_coll.update_one({
                        'HEX_600': row.HEX_600,
                    },
                    {
                        '$push': 
                        {
                            'results': json.loads(decorateRow(row).to_json(orient='records'))[0]
                        }
                    })
                

<ipython-input-13-b50b2f5d2e9e>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')
<ipython-input-13-b50b2f5d2e9e>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')
<ipython-input-13-b50b2f5d2e9e>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')
<ipython-input-13-b50b2f5d2e9e>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')
<ipython-input-13-b50b2f5d2e9e>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')
<ipython-input-13-b50b2f5d2e9e>:34: FutureWarning: The default value of regex wi

In [5]:
# # by subjects
# subject_dir = DATA + "hex_variables_by_subjects_for_viz_system_20220623/"
# for subject in os.listdir(subject_dir):
#     if subject!=".DS_Store":
#         directory = subject_dir + subject + "/"
#         for filename in os.listdir(directory):
#             if filename.endswith(".csv"):
#                 frequency = "all"
#                 if filename.startswith("01"):
#                     user_type = "non_gov"
#                 elif filename.startswith("02"):
#                     user_type = "non_gov_unsure"
#                 elif filename.startswith("03"):
#                     user_type = "all"
#                 elif filename.startswith("04"):
#                     user_type = "non_gov"
#                     frequency = "heavy"
#                 elif filename.startswith("05"):
#                     user_type = "non_gov_unsure"
#                     frequency = "heavy"
#                 elif filename.startswith("06"):
#                     user_type = "all"
#                     frequency = "heavy"
                
#                 data_per_subject = pd.read_csv(os.path.join(directory, filename), header=0, index_col = 0)
#                 data_per_subject.rename(columns={"HEX":"HEX_600","HEX_total_number_of_reports":"HEX_total_reporting"}, inplace=True)

#                 data_per_subject['subject'] = subject
#                 data_per_subject['user_type'] = user_type
#                 data_per_subject['frequency'] = frequency
#                 data = data_per_subject.merge(data_all_subjects, on = 'HEX_600')
#                 data = data.loc[:, ~data.columns.str.contains('_y$')]
#                 data.columns = data.columns.str.replace(r'_x$', '')
#                 for index, row in data.iterrows():
#                     # Append new object from file to existing hexagon's array
#                     hexagon_coll.update_one({
#                         'HEX_600': row.HEX_600,
#                     },
#                     {
#                         '$push': 
#                         {
#                             'results': json.loads(decorateRow(row).to_json(orient='records'))[0]
#                         }
#                     })
                

<ipython-input-5-73716f42e040>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'_x$', '')


In [14]:

for doc in hexagon_coll.find({"HEX_600": 233}):
    for rec in doc['results']:
        print(rec['user_type'])
        print(rec['frequency'])
        print(rec['HEX_weighted_mean_home_distance'])
        print(rec['report_type'])

non_gov
all
1148.3605439858
all
non_gov_unsure
all
1148.3605439858
all
all
heavy
2207.9177425439
all
non_gov
heavy
1373.6741484277
all
all
all
1800.193646956
all
non_gov_unsure
heavy
1373.6741484277
all
all
heavy
136.2255260344
SCHEDULE_A_BULK_ITEM_PICKUP
all
all
136.2255260344
SCHEDULE_A_BULK_ITEM_PICKUP
non_gov_unsure
all
136.2255260344
SCHEDULE_A_BULK_ITEM_PICKUP
non_gov
all
136.2255260344
SCHEDULE_A_BULK_ITEM_PICKUP
non_gov
heavy
136.2255260344
SCHEDULE_A_BULK_ITEM_PICKUP
non_gov_unsure
heavy
136.2255260344
SCHEDULE_A_BULK_ITEM_PICKUP
all
heavy
None
MISSED_TRASH_and_RECYCLING_and_YARD_WASTE_and_BULK_ITEM
all
all
23.2346022109
MISSED_TRASH_and_RECYCLING_and_YARD_WASTE_and_BULK_ITEM
non_gov_unsure
all
23.2346022109
MISSED_TRASH_and_RECYCLING_and_YARD_WASTE_and_BULK_ITEM
non_gov
all
23.2346022109
MISSED_TRASH_and_RECYCLING_and_YARD_WASTE_and_BULK_ITEM
non_gov
heavy
None
MISSED_TRASH_and_RECYCLING_and_YARD_WASTE_and_BULK_ITEM
non_gov_unsure
heavy
None
MISSED_TRASH_and_RECYCLING_and_YAR

In [ ]:
# mongodump --db=boston311